In [1]:
# Import Dependencies

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import tweepy
import time
from tweepy_config import consumer_key, consumer_secret, access_token, access_token_secret

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


In [2]:
# Set up Tweepy credentials

# Twitter API Keys
consumer_key = consumer_key
consumer_secret = consumer_secret
access_token = access_token
access_token_secret = access_token_secret

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [3]:
# Create list of twitter usernames desired for search
news_usernames = ["@BBC","@CBS","@CNN","@FoxNews","@nytimes"]

# Create a list to store individual tweet's compound sentiment
ind_sentiment_list = []

# Create list to hold the final total average of all tweets in the twitter pulls
avg_sentiment_list = []

# Loop through each news agency
for news in news_usernames:

    # Create a set of lists to hold each of the sentiment values
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

    # Paginate through five pages to collect 100 tweets per specified user
    for x in range(1,6):

        # Collect tweets from the specified page for the specified user
        public_tweets = api.user_timeline(news, page=x)

        # Loop through all tweets
        for tweet in public_tweets:

                # Perform VADER analysis and store in variable
                results = analyzer.polarity_scores(tweet["text"])
                compound = results["compound"]
                pos = results["pos"]
                neu = results["neu"]
                neg = results["neg"]

                # Add each value to the compound list so that it can be averaged
                compound_list.append(compound)
                positive_list.append(pos)
                negative_list.append(neg)
                neutral_list.append(neu)
                
                # Also create a dictionary for each individual tweet
                temp_ind_dict = {"User": news,\
                                "Compound":compound,\
                                "Positive":pos,\
                                "Neutral":neu,\
                                "Negative":neg,\
                                }
                
                # Append the individual sentiment list to contain the tweet's information
                ind_sentiment_list.append(temp_ind_dict)
        
        # Use time.sleep to prevent abusing API call
        time.sleep(1)

    # Once all tweets have been gathered for a user, average values gathered on all tweets and add to a dictionary
    sentiment = {
        "User": news,
        "Compound": np.mean(compound_list),
        "Positive": np.mean(positive_list),
        "Neutral": np.mean(negative_list),
        "Negative": np.mean(neutral_list),
        "Tweet Count": len(compound_list)
    }
    
    # Append the average sentiment data to the average sentiment list
    avg_sentiment_list.append(sentiment)
    
    # Print confirmation message for each user
    print(f"Twitter user {news} has been completed")

Twitter user @BBC has been completed
Twitter user @CBS has been completed
Twitter user @CNN has been completed
Twitter user @FoxNews has been completed
Twitter user @nytimes has been completed


In [12]:
# Create pandas DataFrame containing the sentiment analysis of each individual tweet

news_tweet = pd.DataFrame(ind_sentiment_list)

# Reorganize the columns

news_tweet = news_tweet[["User","Compound","Positive","Neutral","Negative"]]

# Display a preview of the DataFrame

news_tweet.head()


,User,Compound,Positive,Neutral,Negative
0,@BBC,-0.6369,0.000,0.682,0.318
1,@BBC,0.0000,0.000,1.000,0.000
2,@BBC,0.0000,0.000,1.000,0.000
3,@BBC,0.1779,0.137,0.758,0.104
4,@BBC,-0.5090,0.000,0.829,0.171


In [14]:
# Create another pandas DataFrame containing the average sentiment analysis for each user

news_avg = pd.DataFrame(avg_sentiment_list)

# Reorganize the columns

news_avg = news_avg[["User","Compound","Positive","Neutral","Negative","Tweet Count"]]

# Display the DataFrame

news_avg


,User,Compound,Positive,Neutral,Negative,Tweet Count
0,@BBC,0.117453,0.09329,0.05148,0.85521,100
1,@CBS,0.248885,0.14218,0.01983,0.83801,100
2,@CNN,0.022574,0.07358,0.06387,0.86255,100
3,@FoxNews,-0.073076,0.06562,0.09511,0.83929,100
4,@nytimes,0.005602,0.07355,0.06723,0.85923,100
